In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import nn
from tensorflow.keras.activations import softmax
from keras import layers
from tensorflow.keras.layers import Dense,LayerNormalization ## alternative for nn.linear
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-12-06 10:55:24.152344: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-06 10:55:24.201400: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 10:55:24.201434: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 10:55:24.202545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 10:55:24.210168: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# X_data = np.load("../../data/data_mfcc.npy")
X_data = np.load("../../data/data_40_500.npy")
X_data = np.transpose(X_data, (0, 2, 1))
# X_data=X_data[:100]
# X_data = np.expand_dims(X_data, axis=-1)
print(X_data.shape)

(13100, 500, 40)


In [3]:
Y_data = np.load("../../data/pad_data_5.npy")
type(Y_data)


numpy.ndarray

In [4]:
def create_self_attention_mask(sequence_length):
    mask = np.tril(np.ones((sequence_length,sequence_length)))
    mask[mask==0]=-np.inf
    mask[mask==1]=0
    return mask

In [5]:
test = create_self_attention_mask(5)
test

array([[  0., -inf, -inf, -inf, -inf],
       [  0.,   0., -inf, -inf, -inf],
       [  0.,   0.,   0., -inf, -inf],
       [  0.,   0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.,   0.]])

In [6]:
def scaled_dot_product(q, k, v, mask):
    d_k = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(d_k)

    if mask is not None:
        scaled_qk += mask

    attention_weights = tf.nn.softmax(scaled_qk)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [7]:
q=[[1.0,2.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
k=[[1.0,3.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
v=[[1.0,1.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
mask_1=create_self_attention_mask(3)
test,weights = scaled_dot_product(q,k,v,mask=mask_1)
print("output is\n",test)
print("attention weights is \n",weights)

2023-12-06 10:55:25.984085: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 10:55:26.013388: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 10:55:26.013710: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

output is
 tf.Tensor(
[[1.         1.         1.        ]
 [0.99999994 0.99999994 0.99999994]
 [1.         1.         1.        ]], shape=(3, 3), dtype=float32)
attention weights is 
 tf.Tensor(
[[1.         0.         0.        ]
 [0.7603684  0.23963155 0.        ]
 [0.6133826  0.19330868 0.19330868]], shape=(3, 3), dtype=float32)


In [8]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_sequence_length):
        super(PositionalEncoding, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    
    def call(self, inputs):
        even_i = tf.range(0, self.d_model, 2, dtype=tf.float32)
        denominator = tf.pow(10000.0, even_i / self.d_model)
        position = tf.reshape(
            tf.range(self.max_sequence_length, dtype=tf.float32),
            (1, self.max_sequence_length, 1),
        )
        even_PE = tf.sin(position / denominator)
        odd_PE = tf.cos(position / denominator)
        stacked = tf.stack([even_PE, odd_PE], axis=2)
        PE = tf.reshape(stacked, (1, self.max_sequence_length, -1))
        print("postional encoding output shape",PE.shape)
        return PE

In [9]:
class ConvolutionalLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape, filters=64, kernel_size=3, **kwargs):
        super(ConvolutionalLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

        self.conv1 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same", trainable=True)
        self.batch_norm1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()

        self.conv2 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same",trainable=True)
        self.batch_norm2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()

        self.global_avg_pooling = layers.GlobalAveragePooling1D()

    
    def call(self, inputs, training=None, mask=None):
        conv1_out = self.relu1(self.batch_norm1(self.conv1(inputs), training=training))
        conv2_out = self.relu2(self.batch_norm2(self.conv2(conv1_out), training=training))
        gap_out = self.global_avg_pooling(conv2_out)
        if training:
            print("CNN output shape is  ", gap_out.shape)
        return gap_out

In [10]:
class Transformer(tf.keras.Model):
    def __init__(self, d_model, max_input_length):
        super(Transformer, self).__init__()
        self.d_model = d_model
        self.cnn_layer = ConvolutionalLayer(input_shape=(None, 500,40))  # Adjust input shape

    def call(self, inputs, training=None):
        cnn_output = self.cnn_layer(inputs)
        return cnn_output


In [11]:
model = tf.keras.Sequential([Transformer(d_model=20,max_input_length=500),layers.Dense(30)])
# model.build(input_shape=(None, 500, 20))

In [12]:
from sklearn.model_selection import train_test_split

# Assuming X_data and Y_data are your input features and labels
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)

print(Y_train.shape)
print(Y_val.shape)

(10480, 500, 40)
(2620, 500, 40)
(10480, 30)
(2620, 30)


In [13]:
optim = tf.keras.optimizers.Adam(learning_rate=10)
model.compile(optimizer=optim, loss='mean_squared_error',metrics=['mae'])
model.build(input_shape=(13100,500,40))  # Replace your_input_shape with the actual input shape
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (Transformer)   (13100, 64)               20608     
                                                                 
 dense (Dense)               (13100, 30)               1950      
                                                                 
Total params: 22558 (88.12 KB)
Trainable params: 22302 (87.12 KB)
Non-trainable params: 256 (1.00 KB)
_________________________________________________________________


In [14]:
def loss_function(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [15]:
model.fit(X_train,Y_train,epochs=10,validation_data=(X_val, Y_val))

Epoch 1/10
CNN output shape is   (None, 64)
CNN output shape is   (None, 64)
CNN output shape is   (None, 64)


2023-12-06 10:55:28.685850: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2023-12-06 10:55:28.752816: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-06 10:55:29.267707: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fb4a82e27a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-06 10:55:29.267731: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-12-06 10:55:29.271960: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1701840329.342711   10574 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


328/328 [==============================] - 5s 9ms/step - loss: 730886.3750 - mae: 485.8378 - val_loss: 662778.1250 - val_mae: 503.3202
Epoch 2/10
328/328 [==============================] - 2s 7ms/step - loss: 653322.1875 - mae: 501.5869 - val_loss: 662669.9375 - val_mae: 506.0793
Epoch 3/10
328/328 [==============================] - 2s 7ms/step - loss: 653340.9375 - mae: 501.6714 - val_loss: 663028.5625 - val_mae: 510.4291
Epoch 4/10
328/328 [==============================] - 2s 7ms/step - loss: 653488.1875 - mae: 502.5520 - val_loss: 663291.6250 - val_mae: 500.9262
Epoch 5/10
328/328 [==============================] - 2s 7ms/step - loss: 653446.7500 - mae: 501.3461 - val_loss: 663269.1250 - val_mae: 512.2100
Epoch 6/10
328/328 [==============================] - 2s 7ms/step - loss: 653586.2500 - mae: 502.1270 - val_loss: 663103.5000 - val_mae: 503.6278
Epoch 7/10
328/328 [==============================] - 2s 7ms/step - loss: 653674.0625 - mae: 502.2512 - val_loss: 663349.3125 - val_mae

In [16]:
prediction = model.predict(X_val)
print("prediction ",prediction[0])
print("actual ",Y_val[0])

82/82 [==============================] - 0s 2ms/step
prediction  [586.49884   818.633     754.5613    736.9478    735.6455    768.6073
 714.1019    682.07367   661.6007    662.4464    610.3579    528.379
 501.02264   498.9172    432.16226   336.79993   303.41092   305.64825
 234.81598   179.41928   174.61377   130.13762    70.81951    71.278625
  36.486122   58.044548   11.78944     1.790735    1.0597305  -4.7081075]
actual  [  10   60 3030    3 3160   40  504  208 1091    6 2561   16 2675    3
 1656  298  117   86 2949   18   33   18   33    0    0    0    0    0
    0    0]
